In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import math
import hvplot.pandas
import holoviews as hv
hv.extension('bokeh')

In [125]:
# [1] is used to select the specific table needed from the Wikipedia website
asx_data = pd.read_html('https://en.wikipedia.org/wiki/S%26P/ASX_200')[1]

In [126]:
asx_data.head(2)

,Code,Company,Sector,Market Capitalisation,Chairperson,HQ
0,A2M,a2 Milk Company,Consumer Staples,4.222573e+09,David Hearn,Auckland
1,ABC,Adbri,Materials,2.114513e+09,Raymond Barro,Adelaide


In [132]:
# Put tickers into a list
tickers = asx_data['Code'].to_list()

In [133]:
# Only need the ['Adj close'] using yfinance
prices = yf.download(tickers, start='2008-01-01', end='2023-05-31')['Adj Close']

[*********************100%%**********************]  200 of 200 completed

39 Failed downloads:
['IVC', 'S32', 'NHF', 'PME', 'AKE', 'GNC', 'SFR', 'APE', 'BRG', 'CCX', 'TLC', 'WBC', 'CXO', 'ABC', 'SQ2', 'A2M', 'TCL', 'MP1']: Exception('%ticker%: No timezone found, symbol may be delisted')
['UWL', 'VUK', 'JBH', 'BKL', 'HDN']: Exception("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")
['STO', 'SGM', 'CHC', 'COH', 'CRN', 'TNE', 'TAH', 'DEG', 'ANN', 'MPL', 'RWC', 'HLS', 'ABP', 'BKW', 'IRE', 'CCP']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2008-01-01 -> 2023-05-31)')


In [134]:
# Change to datetime
prices.index = pd.to_datetime(prices.index)

In [135]:
# Convert from daily to monthly prices using 'M'
prices_monthly = prices.resample('M').last()

In [136]:
# Drop the columns with insufficient data
prices_monthly_clean = prices_monthly.dropna(axis=1)

In [137]:
prices_monthly_clean.head()

Ticker,AIA,ALL,ALX,AMP,APA,ASX,AUB,BAP,BEN,BHP,...,PDN,PNI,PPT,PRU,RIO,RMD,SOL,VEA,WDS,WOR
Date,,,,,,,,,,,,,,,,,,,,,
2008-01-31,31.082825,33.082653,128.790039,38.967979,74.934402,2.147642,12.353293,55.942856,20.415472,32.559647,...,15.137025,5.906592,1.749220,47.862835,36.329906,19.654079,62.549999,27.475292,20.361639,5.334653
2008-02-29,31.011564,32.081173,111.261200,35.677799,90.222687,2.323679,11.304970,57.978703,18.509056,35.542206,...,15.652364,5.253391,1.714140,41.543365,40.870766,17.132265,53.150002,27.196684,25.424770,5.707819
2008-03-31,30.783545,32.578323,130.408630,36.530285,95.028358,2.429301,12.520137,55.958450,19.059605,31.982161,...,15.585383,5.617073,1.703275,44.549385,37.038654,17.847342,56.799999,27.299543,24.226864,5.566879
2008-04-30,34.232433,34.137428,131.070786,33.565228,106.044083,2.595277,12.384401,63.921146,18.698030,39.175106,...,16.135860,5.726694,1.756652,43.103294,42.269241,18.245075,83.750000,28.767338,25.620340,5.943062
2008-05-31,32.992538,34.813915,126.913933,33.402687,105.555885,2.640544,13.089650,66.387001,19.890850,40.962421,...,16.521189,5.736611,1.784575,42.528294,43.438396,16.666826,125.400002,29.114147,30.656393,6.579937


In [138]:
# Use "shift() to move the dataframe up and down relate to the index
momentum_12 = (prices_monthly_clean/prices_monthly_clean.shift(12))-1
monthly_returns = (prices_monthly_clean/prices_monthly_clean.shift(1))-1

In [139]:
# Drop the 1 or 2 rows with NA's
momentum_12 = momentum_12.dropna(axis=0)
monthly_returns = monthly_returns.dropna(axis=0)

In [140]:
# Establish a df for the ranks
quintile_ranks = pd.DataFrame(index=momentum_12.index, columns=momentum_12.columns)

In [141]:
# Place each stock in a "quintile" bucket, at each "date" (rows)
for date in momentum_12.index:
    row_values = momentum_12.loc[date] # looping values each row
    ranks = pd.Series(row_values).rank(method='max') # set rank at each row
    quintiles = pd.qcut(ranks, q=5, labels=False) # divide into quintiles

    # Create dataframe with ranks for each stock at each month(4=high, 0=low)
    quintile_ranks.loc[date] = quintiles

In [142]:
quintile_ranks.head(5)

Ticker,AIA,ALL,ALX,AMP,APA,ASX,AUB,BAP,BEN,BHP,...,PDN,PNI,PPT,PRU,RIO,RMD,SOL,VEA,WDS,WOR
Date,,,,,,,,,,,,,,,,,,,,,
2009-01-31,1,1,1,0,3,0,3,2,1,2,...,2,2,3,0,0,4,0,1,1,2
2009-02-28,2,0,1,0,2,1,3,2,1,2,...,2,3,3,0,0,4,0,1,1,1
2009-03-31,2,0,1,0,1,2,3,2,1,3,...,1,2,3,0,0,4,0,1,1,2
2009-04-30,2,0,1,1,1,2,4,2,2,1,...,2,3,3,0,0,3,0,1,1,4
2009-05-31,2,0,3,1,1,1,2,2,1,2,...,2,3,3,0,0,4,0,1,1,3


In [145]:
quintile_dfs = {} 
portfolio_returns = pd.DataFrame()

# Iterate for each quintile, and form portfolios accordingly
for quintile in range(5):
    
    # Only take returns if they're in quintile associated with the current loop.
    filtered_df = monthly_returns[quintile_ranks == quintile]
    
    # shift to "t+1" return as rank at time "t" corresponds to return at time "t+1"
    filtered_df_shifted = filtered_df.shift(-1).dropna(axis=0)

    # Put those returns into dictionary "quintile_dfs"
    quintile_dfs[quintile] = filtered_df_shifted

    #For Equal-Weight, simply take the average return across stocks within each quintile dataframe
    portfolio_returns[quintile] = quintile_dfs[quintile].mean(axis=1).dropna()


# Plot the cumulative return
(1+portfolio_returns).cumprod().hvplot(title = 'Portfolios - Log Scale', height = 300, width= 500, grid=True, logy=True)

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [147]:
# Calculate Ann Return
cumulative_return = (1 + portfolio_returns).prod() - 1
num_periods = len(monthly_returns)
annualized_return = (1 + cumulative_return) ** (12 / num_periods) - 1

# Calculate Ann Vol
ann_vol = portfolio_returns.std()*math.sqrt(12)

# Calculate Risk-adjusted-return
risk_adj_return = annualized_return/ann_vol
risk_adj_return.hvplot(kind='bar',title = 'Quintile Momentum portfolio - Risk Adjusted Return', height = 300, width= 500, grid=True, color = 'teal')

:Bars   [index]   (0)